HW05 - Problem 4
==============
Performance evaluation. For each of the 3 datasets, train a decision tree and random forest and report your training and validation accuracies. You should be reporting 12 numbers (3 datasets × 2 classifiers × 2 data splits).


Program overhead:

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import numpy as np
import decisiontree as dt
import HW05_utils as ut

Set the base directory for this homework

In [4]:
BASE_DIR = "/Users/mitch/Documents/Cal/2_2017_Spring/COMPSCI 289A - Intro to Machine Learning/HW05/"

Establish a size for the validation set as a fraction of the total training set

In [5]:
valfrac = 0.1

SPAM
---------------------------
Calculate decision tree and random forest training/validation accuracies for the spam dataset.

Begin by importing data, shuffling, and separating into training and validation sets.

In [6]:
# Import
SPAM_PATH = "Data/hw5_spam_dist/spam_data.mat"

spam_data = ut.load_data(SPAM_PATH,BASE_DIR,'training_data')
spam_labels = ut.load_data(SPAM_PATH,BASE_DIR,'training_labels').T
spam_test = ut.load_data(SPAM_PATH,BASE_DIR,'test_data')

In [7]:
# Shuffle
spamdata, spamlabels = ut.shuffle_data(spam_data,spam_labels)

In [8]:
# Separate
spamtraindata,spamvaldata = ut.val_partition(spamdata,valfrac)
spamtrainlabels,spamvallabels = ut.val_partition(spamlabels,valfrac)

Create and train a decision tree classifier using the spam data and labels.

In [217]:
spam_DTclassifier = dt.DecisionTree(treedepth = 100)

In [218]:
spam_DTclassifier.train(spamtraindata,spamtrainlabels)

Calculate the validation accuracy using the trained decision tree

In [220]:
spamDTpredictions = spam_DTclassifier.predict(spamvaldata)

In [221]:
count,total = 0,0
for i in range(len(spamDTpredictions)):
    if spamDTpredictions[i] == spamvallabels[i]:
        count += 1
    total += 1
    
print('Validation Accuracy = %.3f%%'%(100*count/total))

Validation Accuracy = 79.409%


Use this procedure to determine the optimal value for the tree depth hyperparameter.

In [9]:
def testdepth(treedepth,traindata,trainlabels,valdata,vallabels):
    
    DTclassifier = dt.DecisionTree(treedepth)
    DTclassifier.train(traindata,trainlabels)
    predictions = DTclassifier.predict(valdata)
    valAcc = ut.val_accuracy(predictions,vallabels)
    
    return valAcc


In [226]:
for depth in range(1,100,3):
    spamDT_Acc = testdepth(depth,spamtraindata,spamtrainlabels,spamvaldata,spamvallabels)
    print(depth,spamDT_Acc)

1 0.60084388185654
4 0.6814345991561181
7 0.7054852320675106
10 0.7379746835443038
13 0.7713080168776372
16 0.7822784810126582
19 0.7919831223628692
22 0.7911392405063291
25 0.7924050632911392
28 0.7936708860759494
31 0.7945147679324894
34 0.7945147679324894
37 0.7945147679324894
40 0.7940928270042195


KeyboardInterrupt: 

Make decision tree predictions for the test data, and save to a csv file for upload to Kaggle (0-indexing).

In [132]:
spamDTpredictions = spam_DTclassifier.predict(spam_test)
ids = np.arange(len(spamDTpredictions))
spamDTpredictions_csv = np.concatenate(([ids],[spamDTpredictions]),axis=0).T

In [128]:
np.savetxt(BASE_DIR+'spam_DT_testpredictions.csv',spamDTpredictions_csv,fmt='%i',delimiter=',',header='Id,Category',comments='')

Census
---------------------------
Calculate decision tree and random forest training/validation accuracies for the census dataset.

Begin by importing (preprocessed) data, shuffling, and separating into training and validation sets.

In [27]:
# Import
CENSDAT_PATH = "Data/census_traindata_vec.csv"
CENSLBL_PATH = "Data/census_traindata_lbl.csv"
CENSTST_PATH = "Data/census_testdata_vec.csv"

census_data = np.genfromtxt(BASE_DIR+CENSDAT_PATH,delimiter=',')
census_labels = np.genfromtxt(BASE_DIR+CENSLBL_PATH,delimiter=',')
census_test = np.genfromtxt(BASE_DIR+CENSTST_PATH,delimiter=',')

In [20]:
# Shuffle
census_labels = np.reshape(census_labels,(len(census_labels),1))
censusdata, censuslabels = ut.shuffle_data(census_data,census_labels)

In [21]:
# Separate
censustraindata,censusvaldata = ut.val_partition(censusdata,valfrac)
censustrainlabels,censusvallabels = ut.val_partition(censuslabels,valfrac)

Create and train a decision tree classifier using the census data and labels

In [22]:
census_DTclassifier = dt.DecisionTree(treedepth=7)

In [23]:
census_DTclassifier.train(censustraindata,censustrainlabels)

Calculate the validation accuracy using the trained decision tree

In [34]:
censuspredictions = census_DTclassifier.predict(censusvaldata)

In [35]:
censusvalAcc = ut.val_accuracy(censuspredictions,censusvallabels)
    
print('Validation Accuracy = %.3f%%'%(100*censusvalAcc))

Validation Accuracy = 80.165%


Use this procedure to determine the optimal value for the tree depth hyperparameter. (Re-use the function looping over depths in spam)

In [31]:
depthAcc = []
for depth in range(30):
    print('Tree, depth',depth)
    censusDT_Acc = testdepth(depth,censustraindata,censustrainlabels,censusvaldata,censusvallabels)
    depthAcc.append([depth,censusDT_Acc])
    print(censusDT_Acc)

Tree, depth 0
0.7509168704156479
Tree, depth 1
0.7509168704156479
Tree, depth 2
0.7509168704156479
Tree, depth 3
0.7677261613691931
Tree, depth 4
0.777200488997555
Tree, depth 5
0.7885085574572127
Tree, depth 6
0.7976772616136919
Tree, depth 7
0.8001222493887531
Tree, depth 8
0.7982885085574573
Tree, depth 9
0.7979828850855746
Tree, depth 10
0.8016503667481663
Tree, depth 11
0.8001222493887531
Tree, depth 12
0.7964547677261614
Tree, depth 13
0.7924816625916871
Tree, depth 14
0.78820293398533
Tree, depth 15
0.7836185819070904
Tree, depth 16
0.784841075794621
Tree, depth 17
0.781479217603912
Tree, depth 18
0.78239608801956
Tree, depth 19
0.7790342298288508
Tree, depth 20
0.7790342298288508
Tree, depth 21
0.7759779951100244
Tree, depth 22
0.7756723716381418
Tree, depth 23
0.7765892420537898
Tree, depth 24
0.777200488997555
Tree, depth 25
0.7759779951100244
Tree, depth 26
0.7759779951100244
Tree, depth 27
0.7762836185819071
Tree, depth 28
0.7762836185819071
Tree, depth 29
0.776283618581907

Make predictions for the test data, and save to a csv file for upload to Kaggle (1-indexing).

In [38]:
censusDTpredictions = census_DTclassifier.predict(census_test)
ids = np.arange(1,len(censusDTpredictions)+1)
censuspredictions_csv = np.concatenate(([ids],[censusDTpredictions]),axis=0).T

In [39]:
np.savetxt(BASE_DIR+'census_testpredictions.csv',censuspredictions_csv,fmt='%i',delimiter=',',header='Id,Category',comments='')

Titanic
---------------------------
Calculate decision tree and random forest training/validation accuracies for the Titanic dataset.

Begin by importing (preprocessed) data, shuffling, and separating into training and validation sets.

In [40]:
# Import
TITADAT_PATH = "Data/census_traindata_vec.csv"
TITALBL_PATH = "Data/census_traindata_lbl.csv"
TITATST_PATH = "Data/census_testdata_vec.csv"

titanic_data = np.genfromtxt(BASE_DIR+TITADAT_PATH,delimiter=',')
titanic_labels = np.genfromtxt(BASE_DIR+CENSLBL_PATH,delimiter=',')
titanic_test = np.genfromtxt(BASE_DIR+CENSTST_PATH,delimiter=',')

In [41]:
# Shuffle
titanic_labels = np.reshape(census_labels,(len(census_labels),1))
titanicdata, titaniclabels = ut.shuffle_data(titanic_data,titanic_labels)

In [42]:
# Separate
titanictraindata,titanicvaldata = ut.val_partition(titanicdata,valfrac)
titanictrainlabels,titanicvallabels = ut.val_partition(titaniclabels,valfrac)

Create and train a decision tree classifier using the census data and labels

In [43]:
titanic_DTclassifier = dt.DecisionTree(treedepth=8)

In [44]:
titanic_DTclassifier.train(titanictraindata,titanictrainlabels)

Make predictions for the test data, and save to a csv file for upload to Kaggle (1-indexing).

Calculate the validation accuracy using the trained decision tree

In [45]:
titanicpredictions = titanic_DTclassifier.predict(titanicvaldata)

In [46]:
titanicvalAcc = ut.val_accuracy(titanicpredictions,titanicvallabels)
    
print('Validation Accuracy = %.3f%%'%(100*censusvalAcc))

Validation Accuracy = 80.165%


Use this procedure to determine the optimal value for the tree depth hyperparameter. (Re-use the function looping over depths in spam)

In [48]:
depthAcc = []
for depth in range(30):
    print('Tree, depth',depth)
    titanicDT_Acc = testdepth(depth,titanictraindata,titanictrainlabels,titanicvaldata,titanicvallabels)
    depthAcc.append([depth,titanicDT_Acc])
    print(titanicDT_Acc)

Tree, depth 0
0.7683374083129584
Tree, depth 1
0.7683374083129584
Tree, depth 2
0.7894254278728606
Tree, depth 3
0.7894254278728606
Tree, depth 4
0.8028728606356969
Tree, depth 5
0.8068459657701712
Tree, depth 6
0.8099022004889975
Tree, depth 7
0.8117359413202934
Tree, depth 8
0.8123471882640587
Tree, depth 9
0.8105134474327629
Tree, depth 10
0.8089853300733496
Tree, depth 11
0.8040953545232273
Tree, depth 12
0.808679706601467
Tree, depth 13
0.8065403422982885
Tree, depth 14
0.8001222493887531
Tree, depth 15
0.7976772616136919
Tree, depth 16
0.7952322738386308
Tree, depth 17
0.7937041564792175
Tree, depth 18
0.7940097799511002
Tree, depth 19
0.7894254278728606
Tree, depth 20
0.789119804400978
Tree, depth 21
0.7885085574572127
Tree, depth 22
0.7878973105134475
Tree, depth 23
0.7885085574572127
Tree, depth 24
0.7863691931540342
Tree, depth 25
0.7863691931540342
Tree, depth 26
0.7869804400977995
Tree, depth 27
0.7869804400977995
Tree, depth 28
0.7863691931540342
Tree, depth 29
0.785757946

In [36]:
titanicDTpredictions = titanic_DTclassifier.predict(titanic_test)
ids = np.arange(1,len(titanicDTpredictions)+1)
titanicpredictions_csv = np.concatenate(([ids],[titanicDTpredictions]),axis=0).T

In [37]:
np.savetxt(BASE_DIR+'titanic_testpredictions.csv',titanicpredictions_csv,fmt='%i',delimiter=',',header='Id,Category',comments='')